# Dimension Alternating Neural Network

*Before we begin, we need to download the IPIX data files. These files can be found here: http://soma.mcmaster.ca/ipix.php.*

*In this address we find 12 .iso files, which we need to extract to get .cdf files.*

*After we obtain these .cfd files, we need to save them in a folder: '.../datasets/IPIX/15m/cdf' so we can use it later.*

## All Imports

### regular imports

In [1]:
import os
import sys
import glob
import scipy
import shutil
import pickle
import mlflow
import logging
import datetime
import argparse
import commentjson
import numpy as np
from bunch import Bunch
import tensorflow as tf
from pathlib import Path
from random import randint
from collections import OrderedDict
from tensorflow.keras import metrics
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Input
from tensorflow.keras.layers import Dropout,  Activation, LeakyReLU, AveragePooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, Callback, EarlyStopping

### .py imports

In [8]:
from Dafc0 import *
from Dafc1 import *
from Dafc2 import *
from Dafc3 import *
from Dafc4 import *

## Dafc0 - convert .iso to .pkl

Here we take the .cdf files and turn them into workable .pkl files containing all the necessary data:
    
* adc_data (analog to digital converted data) - X: (varies: 27-34, 60000)
* Pulse Repetition Interval - PRI: ()
* Range bins (can be a 1D array or a list) - rng_bins: (varies: 27-34, )
* Bandwidth - B: ()

In [3]:
pol = 'hh'
cdf_dir = '/Users/arigra/Desktop/DL projects/Dafc/datasets/IPIX/15m/cdf'
pkl_dir = '/Users/arigra/Desktop/DL projects/Dafc/datasets/IPIX/15m/pkl1/' + pol
cdf_path_list = [f for f in os.listdir(cdf_dir) if not f.startswith('.')]

for cdf_path in cdf_path_list:

    cdf_path = os.path.join(cdf_dir, cdf_path)
    X, PRI, rng_bins, B = parse_ipix_data(cdf_path, pol=pol, adc_like_I=2, adc_like_Q=3, adc_cross_I=0, adc_cross_Q=1)
    # dump to pkl_file
    dir_name = pkl_dir
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
    pkl_file_name = os.path.basename(cdf_path).replace(".CDF","") + "_pol_" + pol + ".pkl"

    pickle.dump({"PRI": PRI, "rng_bins": rng_bins, "B": B,"adc_data": X}, open(os.path.join(dir_name, pkl_file_name), 'wb'), protocol=3)
print("Successfully created and dumped all pkl files!")

Successfully created and dumped all pkl files!


## Dafc1

Here we set the config file, which determines the specific values and decisions in the code.

We also define the loggings and initialize the GPUs

In [11]:

SRC_DIR = os.getcwd()
config_path = '/Users/arigra/Desktop/DL projects/Dafc/config.json'  
args = get_args(config_path)
config = read_config(args)
gpu_init()
set_logger_and_tracker(config)
print("Successfully loaded config file!")
# To watch the properties of the config object, uncomment the following lines:
#print_config(config)
#config_properties = list(config)
#config_properties.sort()
#for attr_name in config_properties:
#    attr_value = config[attr_name]
#    print(f'{attr_name}: {attr_value}')

Successfully loaded config file!


## Dafc2 - Preprocessing

This is the most complicated part of the code, we will now try to give an explanation as simple as possible, by taking this particular flow.

1. <span style="color: purple;"> **load_data:** We start this with "load_data" which is the main function in this part.</span>

    <span style="color: purple;">In our case,config.data_name = "ipix", therefore "load_data" chooses to use the "get_dataset_ipix" function.</span>

    2. <span style="color: green;">**get_dataset_ipix:** This function operates in 2 different ways, given that config.ipix_cv_mode = False, this is the process:</span>

        <span style="color: green;">We first initialize "data_dict_per_file" so we can store the processed data there, then we now call the "read_ipix_data" function for each file.</span>

        3. <span style="color: orange;">**read_data_ipix:** Here we finally start using the data:</span>
            * <span style="color: orange;">We assign the data from the pkl files to variables (example: PRI = ipix_data['PRI'])</span>
            * <span style="color: orange;">Using these variables, we create the c_tensor (the signal after range steering), clutter velocity, and range bins.</span>
            * <span style="color: orange;">c_tensor:

            $$ \text{omegaClutter} = \frac{2\pi B}{3e8} $$
            $$ \text{clutterRangeSteeringTensor}[i,j] = e^{(-1j \cdot i \cdot \text{omegaClutter}[j])}$$ 
            $$ \text{cTensor} = clutterRangeSteeringTensor \times adcData $$

            * <span style="color: orange;">clutter velocity: we apply the welch method for each range bin. From that, we create the PSD (Power Spectral Density), and by using a simple calculation, we obtain the clutter velocity.</span>
            * <span style="color: orange;">range bins: we set the number of range bins to be the minimum of all samples.</span>
            
            <span style="color: blue;">*finally we obtain the c_tensor with shape (54, 60000), rng_bins with shape (27,), and clutter velocity which is a float64*</span>

        <span style="color: green;"> after we have the c_tensor, clutter velocity and the range bins we define different Ms, which define how many samples of the data we are going to use (for train, validation and test).</span>
        
        <span style="color: green;">Now we want to define the train, validation and test sets, in order to do so, we use the "gen_ipix_pipeline_dataset" function.</span>
        
        4. <span style="color: orange;">**gen_ipix_pipeline_dataset:** 
        
            <span style="color: orange;">This function processes the ipix dataframes, applies random Doppler shifts, generates radar signals and labels for target detection and returns a dataset consisting of frames with and without targets.</span>


            5. <span style="color: red;"> **get_reconstruction_point_cloud_vec:** </span>
            
                <span style="color: red;">calculates and returns vectors used for point cloud reconstruction in the ipix pipeline.
                the function first checks if we use FFT dimensions for point cloud reconstruction, If this option is enabled, it rescales the dimensions(in our case it is enabled). This rescaling is done to achieve higher resolution in the reconstructed point cloud.</span>

                <span style="color: red;">The function then calls the get_fft_resolutions function to calculate the resolution and values for the range, velocity, and azimuth dimensions of the point cloud.</span>

                <span style="color: red;">If param_ind is 0, it returns the range bins values, which represent the distances from the radar sensor to the objects in the range dimension. If param_ind is 1, it returns the velocity bins values, representing the velocities of the objects.</span>
            
            <span style="color: blue;">finally, we are left with recon_vec_rng (27,) and recon_vec_vel (63,)</span>
            
            <span style="color: orange;">now, after we obtaind the recon vectors, we get tfds0 and tfds1 with the get_ipix_tfds function, which maps the in the following way:</span>

            6. <span style="color: red;">**gen_ipix_frame2d**</span>

                <span style="color: red;">This function generates a single frame of the ipix pipeline in the following way:</span>

                 * <span style="color: blue;">randomly cropping the c_tensor, making it a tensor of size (54,64)</span>
                 * <span style="color: red;">performing doppler shifts</span>
                 * <span style="color: red;">for targetless (tfds0), fills zeros and -1000.0 values to indicate no target</span>
                 * <span style="color: red;">for tfds1, using the "gen_target_matrix" function to generate the needed data </span>
                    
                          

                7. <span style="color: yellow;">**get_target_matrix:**</span>

                    <span style="color: yellow;">Generates a target matrix along with corresponding labels, parameter values,and SCNR value.</span>
                
                <span style="color: red;">The output of this function is:</span>

                <span style="color: blue;">rd_signal + c_tensor (54,64), label_tensor (27,63), param_val_tensor(None,), scnr_tensor(None,), tf.constant(0.0), clutter_vel_local(), tf.constant(0.0)</span>

                <span style="color: red;">so, tfds = return rd_signal + c_tensor, label_tensor, param_val_tensor, scnr_tensor, tf.constant(0.0), clutter_vel_local, tf.constant(0.0)</span>

                <span style="color: red;">We now concatenate the tfds with (tfds1) and without targets (tfds0), and use the split_auxillary_structure to map them, the tfds size loops between 555, 55 and 740 for train, test and validation</span>

        <span style="color: green;">We are now finally back to get_dataset_ipix. Testing the properties of the train, validation and test we get a dictionary of 740 train set _mapDataset, 555 test set _mapDataset, and 55 validation set _mapDataset</span>

        <span style="color: green;">Now, on each one we apply a tensorFlow preprocess using the function:</span>

        8. <span style="color: orange;">**"tf_dataset_pipeline:"**</span> 

            * <span style="color: orange;">transpose the data, swapping real and imaginary parts of each element.</span> 
            * <span style="color: orange;">center by subtracting the mean from the last axis and reshape to original shape.</span> 
            * <span style="color: orange;">standartization by element-wise division.</span> 
            * <span style="color: orange;">concat the real and imaginary parts of the matrix along the last axis</span> 
            * <span style="color: orange;">processing the labels.</span> 
        
        <span style="color: green;">Which gives us the data processed as we need.</span>

    <span style="color: purple;">We now finally activate the functions "get_model_input_dim" and "get_model_output_dim" to get the input and output dimensions.</span>

    <span style="color: purple;">We also use the "make_iterators" function to prepare iterators for this dataset, which can be used for efficient batch processing.</span>



        



   


               


    

        


In [5]:
config, data = load_data(config)
print("Successfully loaded data!")

make_iterators(): M_train: 19988
Successfully loaded data!


## Dafc3 - Building the model

Here we build the model, Using the settings in the config file.



In [6]:
model = build_model(config)

Model: "TwoStageFcModel"
____________________________________________________________________________________________________________________________________________
 Layer (type)                                                  Output Shape                                            Param #              
 input (InputLayer)                                            [(None, 54, 128)]                                       0                    
                                                                                                                                            
 two_stage_fc_layer (TwoStageFcLayer)                          (None, 128, 1024)                                       139136               
                                                                                                                                            
 two_stage_fc_layer_1 (TwoStageFcLayer)                        (None, 16, 256)                                         264464    

## Dafc4

In [9]:
trainer = build_trainer(model, data, config)
history = trainer.train()

Epoch 1/300


KeyboardInterrupt: 